# Agent Supervisor Architecture Application
Applying a supervisor approach to ideas of social media sentiment analysis

---

<img src="agent_supervisor_diagram.png" width=400/>

---

## Dependencies

In [1]:
import os
from langchain_core.tools import tool
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
import functools
from langgraph.graph import StateGraph, END
import operator

---

## Gathering API keys

OpenAI and LangChain API key set in environment variable, `langchain_tracing_v2` allows langsmith agent tracing, `langchain_project` determines organization in langsmith

In [2]:
# openai key, langchain key all environment variables

os.environ["LANGCHAIN_TRACING_V2"] = "True"
os.environ["LANGCHAIN_PROJECT"] = "Social Collab"

---
# Defining Tools for Agents to use

*Emotion Analysis Tool*

This tool uses a fine tuned version of **GPT-3.5-Turbo** tuned on the [Go Emotions Hugging Face dataset](https://huggingface.co/datasets/go_emotions)  

It takes a string input and returns one of 28 Emotion labels.

In [3]:
emotion_analysis_template = """
You are a cutting edge emotion analysis classification assistant.\
You analyze a comment, and apply one or more emotion labels to it. \

The emotion labels are detailed here: \

['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']

Your output should simply be just the respective emotion, and if there are multiple seperated with commas. \

The comment is here: {comment}
"""

output_parser = StrOutputParser()
emotion_llm = ChatOpenAI(temperature=0.0, model="ft:gpt-3.5-turbo-0125:personal:go-emotions:95jDha5f")
emotion_analysis_prompt = ChatPromptTemplate.from_template(emotion_analysis_template)

emotion_chain = (
    {"comment": RunnablePassthrough()} 
    | emotion_analysis_prompt
    | emotion_llm
    | output_parser
)

@tool
def analyze_emotion(query: str) -> str:
    """Analyze the emotion of a string of text"""
    emotion = emotion_chain.invoke(query)
    return emotion

*Sentiment Analysis Tool*

This tool uses a fine tuned version of **GPT-3.5-Turbo** on the [Tweet Sentiment Extraction Hugging Face Dataset](https://huggingface.co/datasets/mteb/tweet_sentiment_extraction)

It takes in a string input and returns either Postivie, Neutral, or Negative

In [4]:
# Sentiment Detecting Tool Definition
sentiment_analysis_template = """
You are a cutting edge emotion sentiment classification assistant.\
You analyze a social media comment, and apply one sentiment label to it. \
The sentiment labels are simple positive, neutral, and negative.
Your output should simply be just the respective sentiment. \

The comment is here: {comment}
"""

output_parser = StrOutputParser()
ft_gpt35t_llm = ChatOpenAI(temperature=0.0, model="ft:gpt-3.5-turbo-0125:personal:twitter-sentiment:97Cp4jlV")
sentiment_analysis_prompt = ChatPromptTemplate.from_template(sentiment_analysis_template)

sentiment_chain = (
    {"comment": RunnablePassthrough()} 
    | sentiment_analysis_prompt
    | ft_gpt35t_llm
    | output_parser
)

@tool
def analyze_sentiment(query: str) -> str:
    """Analyze the sentiment of a string of text"""
    sentiment = sentiment_chain.invoke(query)
    return sentiment

*Report Generator Tool*

This tool is a prompt chain using **GPT-4-Turbo** that takes in information about social media comments and generates a formatted report with analysis and takeaways.

In [5]:
# Report Gen Tool Definition
report_template = """
You are a social media specialist going through and performing routine analysis of social media community reaction. \
Specifically, you are analyzing the comments of one of your company's social media posts in order to improve strategy and ideas for future posts. \
Provided is a list of comments and replies for the current social media post. \
Your intent is to provide the overall reaction to this post, and the discussion that ensued from it as a first part, 
And then discuss specific examples and reccomendations that can used to create new posts that are tailored towards the positive sentiments and emotions, and away from the negative ones. \
Provide examples if useful to the analysis. As you have limited time to present, keep this report to 500 words or less.\
It is critical to be brief and to the point in your social media strategy reccomendations, providing your actionable takeaways and insights as clearly as possible. Ensure that your ideas are unique, realistic, and not broad or boring.

Context is here: {context}
"""

output_parser = StrOutputParser()
report_llm = ChatOpenAI(temperature=0.0, model="gpt-4-0125-preview")
report_prompt = ChatPromptTemplate.from_template(report_template)

report_chain = (
    {"context": RunnablePassthrough()} 
    | report_prompt
    | report_llm
    | output_parser
)

@tool
def generate_report(query: str) -> str:
    """Input all robust information about social media data and generate a draft of a report. The more input the better the draft."""
    report = report_chain.invoke(str(query))
    with open("generated_report.txt", 'w') as file:
        file.write(report)
    return report

*Report Revision Tool*

This tool is a prompt chain using **GPT-4-Turbo** that takes a second look at a generated report to either reformat or add additional thoughts.

In [6]:
# Revision Definition
revision_template = """
You are a social media specialist helping with the revision of a report. Please analyze and revise this report, ensuring it is, \
professional, specific, and elaborate on any interesting ideas to provide more assistance in helping someone ideate through ideas. \

Context is here: {context}
"""

output_parser = StrOutputParser()
revision_llm = ChatOpenAI(temperature=0.0, model="gpt-4-0125-preview")
revision_prompt = ChatPromptTemplate.from_template(revision_template)

revision_chain = (
    {"context": RunnablePassthrough()} 
    | revision_prompt
    | revision_llm
    | output_parser
)

@tool
def revise_report(query: str) -> str:
    """Take a report, analyze, and refine it for a final output."""
    report = report_chain.invoke(str(query))
    with open("revised_report.txt", 'w') as file:
        file.write(report)
    return report

---
# Creating Agents

### Helper Functions for agent creation

Input an LLM as a `ChatOpenAI` object, available tools as a list, and system prompt as a string, and it will format into the correct format.

This also includes message placeholders for "messages," or what will be a a list of operations that have happened, and `agent_scratchpad` which acts as the agent's specific memory.

The `agent_node` helper will assist in creating the node in the overall agent graph, invoking the agent with the current state of the operation, and outputing a `HumanMessage` object with the output to be used as input into the next node if required

In [7]:
# Helper Functions
def create_agent(
    llm: ChatOpenAI, tools: list, system_prompt: str
):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

## Creating The Main Agent Supervisor

Explicit formatting of the main "supervisor" that will be facilitating the "conversation" between the two sub agent nodes, Analyzer and Writer. Possible output actions are the `members` + `FINISH` to determine where to route or to end in the agent loop. This is set up as an OpenAI function Schema, along with a system prompt properly formatted. Partial here dynamically inserts the options and members into the prompt. 

Supervisor chain combines the prompt, binds the route function as a tool option to the LLM, with a JSON output parser that helps with interpreting the function calls.

In [8]:
members = ["Analyzer", "Writer"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers: {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a task and"
    " respond with their results and status. When finished, respond with FINISH."
)
# Our team supervisor is an LLM node. It just picks the next agent to process and decides when the work is completed
options = ["FINISH"] + members
# using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who would act next? Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

llm = ChatOpenAI(model="gpt-4-1106-preview")

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

---

# Creating the Graph

**AgentState Class**: Defines the structure of the "state" that will be passed between agents in the system. `BaseMessage` operators are added in sequence, effectively tracking the workflow as an ongoing conversation or data exchange between agents. `next` is a string that specifies the next agent or node in the workflow to route the process to.

**Analysis Agent Setup**: An instance of GPT-4-Turbo is made, then the helper function is invoked with the LLM object, a list of the tools it intends to use `[analyze_emotion, analyze_sentiment]`, and a system prompt. The node uses a partial function of `agent_node` with the agent and name preset, so the node only needs to take in the state as defined in the helper function `agent_node` above.

**Writer Agent Setup**: An instance of GPT-3.5-Turbo is made, then the helper function is invoked with the LLM object, a list of tools it intends to use `[generate_report, revise_report]`, and a system prompt. The node is then defined with a partial function of `agent_node` in the same way the analysis agent is made.

In [9]:
# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str

analysis_llm = ChatOpenAI(model="gpt-4-1106-preview")
analysis_agent = create_agent(analysis_llm, [analyze_emotion, analyze_sentiment], "You are a social media analysis assistant. You thoroughly analyze emotion and sentiment of provided social media data.")
analysis_node = functools.partial(agent_node, agent=analysis_agent, name="Analyzer")

write_llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
writer_agent = create_agent(write_llm, [generate_report, revise_report], "You generate robust, helpful, and accurate documents, and look over them at least once more for revision before returning the final output.")
writer_node = functools.partial(agent_node, agent=writer_agent, name="Writer")

### Workflow Setup

A `StateGraph` is initialized with the `AgentState` class structure. This graph represents the entire workflow of the system, where nodes represent agents and edges represent the possible transitions based on the agents' outputs.
Nodes are added with the `add_node` function using the above defined nodes and a name, with the supervisor node being the above defined chain.

In [10]:
workflow = StateGraph(AgentState)
workflow.add_node("Analyzer", analysis_node)
workflow.add_node("Writer", writer_node)
workflow.add_node("supervisor", supervisor_chain)

### Defining Edges

**Edges** define the logic of which node, or which agent can "communicate" with others. The initial for loop connects all of the members to the `supervisor` as they will all need to report back to the supervisor when finished for a supervisor and sub agent architecture that we defined in the graph at the top of the notebook.

**Conditional Edges** are set up first using a conditional map, which is a dictionary created with the keys from `members` and an additional key for `FINISH`. Each member maps to itself meaning if the `next` field in the graph state indicates a member's name, the workflow will route to that member's node. The `FINISH` key maps to `END` which terminates the agent loop. 

To put it together, the `add_conditional_edges` method adds edges to the supervisor node that are conditional based on the value of the `next` field in the graph state. The lambda function extracts the `next` field from the state, and the `conditional_map` is used to determine the target node based on the value. If `next` is a members name the workflow routes to that member. If it's `FINISH` the workflow ends.

In [11]:
for member in members:
    # We want our workers to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "supervisor")
# The supervisor populates the "next" field in the graph state which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Finally, add entrypoint
workflow.set_entry_point("supervisor")

graph = workflow.compile()

---

# Trying it out!

In [8]:
import json

file_path = "/Users/alucek/Documents/Jupyter_Notebooks/agent_testing/comments_short.json"
with open(file_path) as f:
    comments =  json.load(f)

for s in graph.stream(
    {
        "messages": [
            HumanMessage(content=f"First perform robust analysis and then secondly write up a report on these social media comments: {comments}")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("------")

{'supervisor': {'next': 'Analyzer'}}
------
{'Analyzer': {'messages': [HumanMessage(content='# Social Media Comments Analysis Report\n\n## Overview\nThis report provides a comprehensive analysis of the emotions and sentiments expressed in a series of social media comments regarding innovation and leadership.\n\n## Comment Analyses\n\n1. Emotion: Neutral\n   Sentiment: Neutral\n   *Summary*: The comment suggests that frustration, rather than necessity, is often the true driver of innovation.\n\n2. Emotion: Neutral\n   Sentiment: Negative\n   *Summary*: The comment emphasizes the importance of a supportive leadership environment for innovation and criticizes stifling, micromanaging leadership.\n\n3. Emotion: Neutral\n   Sentiment: Neutral\n   *Summary*: The commenter expresses concern over the lack of integration of proven technologies into new fields and suggests maximizing existing resources.\n\n4. Emotion: Curiosity\n   Sentiment: Neutral\n   *Summary*: Sharing a personal story relate

# Report

---

## Recommendations for Future Social Media Posts

### Tailoring Towards Positive Sentiments

1. **Leadership Empowerment Series**: Create a series of posts featuring stories of how leadership within various industries has empowered their teams to innovate. Highlight specific strategies and outcomes to showcase the positive impact of supportive leadership.

2. **Innovation Case Studies**: Share real-world examples of companies or teams that successfully integrated existing technologies into new fields. Focus on the process, the challenges overcome, and the benefits of maximizing resources.

3. **Creativity and Autonomy Spotlights**: Develop content around teams or projects where autonomy and creativity were given center stage. Include interviews, behind-the-scenes looks, and the innovative results achieved. This aligns with the admiration for autonomy and the belief in creativity as enablers of innovation.

4. **Risk-Taking and Innovation**: Publish posts that discuss the importance of taking risks in the innovation process. Use quotes from industry leaders, historical examples, and current success stories to illustrate the concept.

### Addressing Neutral to Negative Sentiments

1. **Overcoming Frustration in Innovation**: Address the idea that frustration can drive innovation by sharing tips and strategies for turning frustration into a positive force. Include expert advice and motivational stories to inspire.

2. **Breaking Free from Micromanagement**: Create content that offers solutions for teams struggling under micromanaging leadership. Provide actionable steps for leaders to foster a more open, trusting environment.

3. **The Role of Training in Innovation**: Acknowledge the necessity of not just having diverse teams and tools but also proper training. Develop posts that offer insights into effective training methods that enhance innovation capabilities.

### Unique and Realistic Ideas

- **Interactive Innovation Challenges**: Launch a series of interactive posts that challenge followers to come up with innovative solutions to hypothetical industry problems. Offer rewards for the most creative or practical solutions, fostering a community-driven approach to innovation.

- **Leadership AMA (Ask Me Anything) Sessions**: Host AMA sessions with leaders known for their innovative approaches. This provides followers with direct access to thought leaders and reinforces the positive sentiment towards empowering leadership.

- **Behind-the-Innovation Stories**: Share stories that go behind the scenes of major innovations, focusing on the emotional journey, the role of leadership, and the team dynamics. This humanizes the innovation process and aligns with the positive sentiments around team empowerment and strong leadership.

### Conclusion

By focusing on content that highlights leadership empowerment, creativity, autonomy, and the practical integration of existing knowledge, the company can align its social media strategy with the positive sentiments expressed by its community. Addressing the concerns around micromanagement and the need for proper training, while also offering interactive and engaging content, will further enhance the company's social media presence and foster a more innovative community.

---

# [LangSmith Run Trace](https://smith.langchain.com/public/b6682759-8e00-494e-952a-1c29b069f6ed/r)